In [1]:
# !/usr/bin/python3
import pandas as pd
import scipy as sp
import numpy as np
import matplotlib
from rpy2.robjects.packages import importr
import rpy2.robjects as ro
import sys
if sys.version_info[0] < 3:
    import Tkinter
else:
    import tkinter
from tkinter import messagebox
from tkinter import filedialog
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg, NavigationToolbar2TkAgg
from matplotlib.backend_bases import key_press_handler
from matplotlib.figure import Figure
import sys
import os
%matplotlib inline

#####################################################################################
def path_not_found():
    msg = messagebox.showinfo( "Please select a directory !!!")
#####################################################################################
def select_directory():
    directory = filedialog.askdirectory()
    var.set("PATH: "+directory)
    f = open("textpath.txt", 'w')
    f.write(directory)
#####################################################################################
def show_message():
    f = open("text2.txt",'r')
    s = f.read()
    msg = messagebox.showinfo( "Time Series Classification", "Input time series is: "+s)
#####################################################################################
def done_saving():
    msg = messagebox.showinfo( "Success", "Forecasted values stored in : "+os.getcwd())
######################################################################################
def forecastmethod():
    actual_path = os.getcwd()
    f = open("textpath.txt", "r")
    y = f.read()
    dummy = y.split("/")
    if(len(dummy)<=2):
        path_not_found()
        return
    directory = y
    os.chdir(y)
    importr("tsintermittent")
    ro.r('''directory <- getwd()
            allfilespath = list.files(directory, full.names = TRUE)
            len <- length(allfilespath)
            range <- 1:len
            for(i in range)
            {
                presentfile <- read.csv(allfilespath[i])
                names(presentfile) = c("id","demandvalues")
                arr <- presentfile$demandvalues
                z = idclass(t(arr),type='SBC')
                if(z$cv2 <= 0.49 && z$p <=1.32)
                {
                    output="Smooth"
                }
                else if(z$cv2 > 0.49 && z$p <=1.32)
                {
                    output="Erratic"
                }
                else if(z$cv2 <= 0.49 && z$p >1.32)
                {
                    output="Intermittent"
                }
                else
                {
                    output="Lumpy"
                }
            }
            print (output)
        ''')
    os.chdir(actual_path)
    ro.r('write(output, "text2.txt")')
    show_message()
######################################################################################CROSTON
def croston():
    actual_path = os.getcwd()
    f = open("textpath.txt", "r")
    y = f.read()
    dummy = y.split("/")
    if(len(dummy)<=2):
        path_not_found()
        return
    directory = y
    os.chdir(y)
    forecast = importr("forecast")
    ro.r('''directory <- getwd()
            allfilespath = list.files(directory, full.names = TRUE)
            final_dataframe = data.frame()
            len <- length(allfilespath)
            range <- 1:len
            dat = data.frame()
            for(i in range)
            {
                presentfile <- read.csv(allfilespath[i], sep='\t')
                names(presentfile) = c("id", "demandvalues")
                arr <- presentfile$demandvalues
                ar1 <- croston(arr)
                tempdataframe = data.frame(ar1)
                cname = paste("csv",i, sep="")
                names(tempdataframe) = c(cname)
                newdf <- t(tempdataframe)
                dat <- rbind(dat, newdf)
            }
        ''')
    ro.r('finaldf <- t(dat)')
    ro.r('write.csv(finaldf, file = "Croston-Output.csv")')
    done_saving()
    os.chdir(actual_path)
    var.set("PATH: "+os.getcwd())
######################################################################################HOLTWINTERS
def holtwinters():
    actual_path = os.getcwd()
    f = open("textpath.txt", "r")
    y = f.read()
    dummy = y.split("/")
    if(len(dummy)<=2):
        path_not_found()
        return
    directory = y
    os.chdir(y)
    alpha=0.2
    beta=0.1
    gamma=0.05
    c=1
    for root,dirs,files in os.walk(directory):
        i = 0
        for fi in files:
            i = i+1
            final_df = pd.DataFrame()
            file = os.path.join(directory, fi)
            with open(file, 'r') as f:
                df = pd.read_csv(f, sep='\t')
                df.columns = ['id', 'frequency', 'demandvalues']
                y11 = df['demandvalues']
                y = y11.values
                length = len(y)
                c = int(length/2)
                ylen =len(y)
                if ylen % c !=0:
                    return None
                fc =float(c)
                ybar2 =sum([y[i] for i in range(c, 2 * c)])/ fc
                ybar1 =sum([y[i] for i in range(c)]) / fc
                b0 =(ybar2 - ybar1) / fc
                #Compute for the level estimate a0 using b0 above.
                tbar  =sum(i for i in range(1, c+1)) / fc
                a0 =ybar1  - b0 * tbar 
                #Compute for initial indices
                I =[y[i] / (a0 + (i+1) * b0) for i in range(0, ylen)]

                S=[0] * (ylen+ c)
                for i in range(c):
                    S[i] =(I[i] + I[i+c]) / 2.0

                #Normalize so S[i] for i in [0, c)  will add to c.
                tS =c / sum([S[i] for i in range(c)])
                for i in range(c):
                    S[i] *=tS
                # Holt - winters proper ...
                F =[0] * (ylen+ c)   
                At =a0
                Bt =b0
                for i in range(ylen):
                    Atm1 =At
                    Btm1 =Bt
                    At =alpha * y[i] / S[i] + (1.0-alpha) * (Atm1 + Btm1)
                    Bt =beta * (At - Atm1) + (1- beta) * Btm1
                    S[i+c] =gamma * y[i] / At + (1.0 - gamma) * S[i]
                    F[i]=(a0 + b0 * (i+1)) * S[i]
                l12 = []
                for m in range(c):
                    l12.append((At + Bt* (m+1))* S[ylen + m])
                a = np.array(l12)
                df1 = pd.Series(a)
                final_df[i]=df1
        final_df.to_csv('Holtwinters-Output.csv', sep='\t')
    done_saving()
    os.chdir(actual_path)
    var.set("PATH: "+os.getcwd())
######################################################################################ARIMA
def arima():
    actual_path = os.getcwd()
    f = open("textpath.txt", "r")
    y = f.read()
    dummy = y.split("/")
    if(len(dummy)<=2):
        path_not_found()
        return
    directory = y
    os.chdir(y)
    importr("forecast")
    importr("tseries")
    ro.r('''
        directory <- getwd()
        allfilespath = list.files(directory, full.names = TRUE)
        final_dataframe = data.frame()
        len <- length(allfilespath)
        range <- 1:len
        dat = data.frame()
        for(i in range)
        {
            presentfile <- read.csv(allfilespath[i], sep='\t')
            names(presentfile) = c("id","demandvalues")
            arr <- presentfile$demandvalues
            count_ts = ts(arr)
            clean_cnt = tsclean(count_ts)
            cnt_ma = ma(clean_cnt, order=1)
            count_ma = ts(na.omit(cnt_ma), frequency=7)
            decomp = stl(count_ma, s.window="periodic")
            deseasonal_cnt <- seasadj(decomp)
            fit <- auto.arima(deseasonal_cnt,start.p=1,start.q=1,max.p = 5,max.q = 5,max.P = 5,max.Q = 5,max.d = 3,seasonal = TRUE,ic = 'aicc')
            x = forecast(fit,h=20)
            yy = as.data.frame(x)
            filename = paste("Arima",i,sep="")
            csvfilename = paste(filename,"csv",sep=".")
            write.csv(yy,csvfilename)
        }
    ''')
    done_saving()
    os.chdir(actual_path)
    var.set("PATH: "+os.getcwd())
#####################################################################################
top = tkinter.Tk()
top.geometry("800x500")
os.chdir("D:\Honors\Forecasting Algorithms")

hw = tkinter.Button(top, text = "Holt-Winters", command = holtwinters, bg="skyblue")
hw['font'] = "Verdana 10"
hw.place(x = 10,y = 100, width=100)

arima = tkinter.Button(top, text = "Arima", command = arima, bg="pink")
arima['font'] = "Verdana 10"
arima.place(x = 10, y=150, width=100)

croston = tkinter.Button(top, text = "Croston", command = croston, bg="orange")
croston['font'] = "Verdana 10"
croston.place(x = 10, y=200, width=100)

select = tkinter.Button(top, text = "Browse", command = select_directory)
select['font'] = "Verdana 15"
select.place(x = 650,y = 44, width=100)

var = tkinter.StringVar()
label = tkinter.Label( top, textvariable = var, bg = "white",font = "Verdana 10 bold")
label.place(x = 10, y=50, width=610, height=25)
var.set("PATH: "+os.getcwd())

var1 = tkinter.StringVar()
label = tkinter.Label( top, textvariable = var1, bg = "lime",font = "Verdana 10 bold")
label.place(x = 10, y=250, width=100, height=25)

croston = tkinter.Button(top, text = "Best Forecast-method", command = forecastmethod)
croston['font'] = "Verdana 10"
croston.place(x = 150, y=250, width=300)

top.title("Forecasting Tool")
top["bg"]="grey"
top.mainloop()

In [17]:
# !/usr/bin/python3
from tkinter import *

root = Tk()

var = StringVar()
label = Label( root, textvariable = var, relief = RAISED )

var.set("Hey!? How are you doing?")
label.pack()
root.mainloop()

In [16]:
import matplotlib
matplotlib.use('TkAgg')

from numpy import arange, sin, pi
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg, NavigationToolbar2TkAgg
from matplotlib.backend_bases import key_press_handler


from matplotlib.figure import Figure

import sys
if sys.version_info[0] < 3:
    import Tkinter as Tk
else:
    import tkinter as Tk

root = Tk.Tk()
root.wm_title("Embedding in TK")


f = Figure(figsize=(5, 4), dpi=100)
a = f.add_subplot(111)
t = arange(0.0, 3.0, 0.01)
s = sin(2*pi*t)

a.plot(t, s)


# a tk.DrawingArea
canvas = FigureCanvasTkAgg(f, master=root)
canvas.show()
canvas.get_tk_widget().pack(side=Tk.TOP, fill=Tk.BOTH, expand=1)

toolbar = NavigationToolbar2TkAgg(canvas, root)
toolbar.update()
canvas._tkcanvas.pack(side=Tk.TOP, fill=Tk.BOTH, expand=1)


def on_key_event(event):
    print('you pressed %s' % event.key)
    key_press_handler(event, canvas, toolbar)

canvas.mpl_connect('key_press_event', on_key_event)

def _quit():
    root.quit()     # stops mainloop
    root.destroy()  # this is necessary on Windows to prevent
                    # Fatal Python Error: PyEval_RestoreThread: NULL tstate

button = Tk.Button(master=root, text='Quit', command=_quit)
button.pack(side=Tk.BOTTOM)

Tk.mainloop()

C:\Users\MY PC\Anaconda3\lib\site-packages\matplotlib\__init__.py:1405: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


In [20]:
actual_path = os.getcwd()
os.chdir("D:\Honors\Forecasting Algorithms\Croston Implemntation(DataSet, Output, Code)\Croston")
importr("forecast")
importr("tseries")

ro.r('''
        directory <- getwd()
        allfilespath = list.files(directory, full.names = TRUE)
        final_dataframe = data.frame()
        len <- length(allfilespath)
        range <- 1:len
        dat = data.frame()
        for(i in range)
        {
            presentfile <- read.csv(allfilespath[i], sep='\t')
            names(presentfile) = c("id", "frequency","demandvalues")
            arr <- presentfile$demandvalues
            count_ts = ts(arr)
            clean_cnt = tsclean(count_ts)
            cnt_ma = ma(clean_cnt, order=4)
            count_ma = ts(na.omit(cnt_ma), frequency=7)
            decomp = stl(count_ma, s.window="periodic")
            deseasonal_cnt <- seasadj(decomp)
            fit <- auto.arima(deseasonal_cnt,start.p=1,start.q=1,max.p = 5,max.q = 5,max.P = 5,max.Q = 5,max.d = 3,seasonal = TRUE,ic = 'aicc')
            x = forecast(fit,h=20)
            yy = as.data.frame(x)
            filename = paste("Arima",i,sep="")
            csvfilename = paste(filename,"csv",sep=".")
            write.csv(yy,csvfilename)
        }
 ''')
os.chdir(actual_path)